In [2]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import time
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import os
# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path

from google.colab import drive 
drive.mount('/content/drive/')
%cd "drive"
%cd "MyDrive"
%cd "Ex4Data"
!ls
import import_ipynb
import utils_for_hackathon as utils


Mounted at /content/drive/
/content/drive
/content/drive/MyDrive
/content/drive/.shortcut-targets-by-id/1o0YBS756Yf3fr4s6rHsprRqjOfdY6PDU/Ex4Data
6xw6.fasta		NbTestSet		 our_pred.pdb
6xw6.pdb		net_for_hackathon.ipynb  __pycache__
first_model.tf		net.ipynb		 second_model.tf
fourth_model.tf		our_6xw62.pdb		 third_model.tf
input_data.npy		our_6xw63.pdb		 train_input.npy
label_data.npy		our_6xw64.pdb		 train_labels.npy
model_hackathon.tf	our_6xw65.pdb		 utils_for_hackathon.ipynb
model_loss_history.png	our_6xw6.pdb		 utils.ipynb
my_model.tf		our_model.tf
importing Jupyter notebook from utils_for_hackathon.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################


# number of ResNet blocks for the first ResNet and the kernel size.
RESNET_1_BLOCKS = 3 
RESNET_1_KERNEL_SIZE = 15
RESNET_1_KERNEL_NUM = 64


###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################


# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 3
RESNET_2_KERNEL_SIZE = 3
RESNET_2_KERNEL_NUM = 32
DILATION = [1, 2, 4, 8]

# percentage of dropout for the dropout layer
DROPOUT = 0.15

# number of epochs, Learning rate and Batch size
EPOCHS = 60
LR = 0.01
BATCH = 32

In [ ]:
def resnet_1(input_layer):  # TODO: implement this!
    """
    ResNet layer - input -> BatchNormalization -> Conv1D -> Relu -> BatchNormalization -> Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for _ in range(RESNET_1_BLOCKS):
      out_batchnorm = layers.BatchNormalization()(input_layer)
      out_conv1 = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same', activation='relu')(out_batchnorm)
      out_batchnorm = layers.BatchNormalization()(out_conv1)
      out_conv1 = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same', activation='relu')(out_batchnorm)
      input_layer = layers.Add()([input_layer, out_conv1])
    return input_layer





In [ ]:
def resnet_2(input_layer):  # TODO: implement this!
    """
    Dilated ResNet layer - input -> BatchNormalization -> dilated Conv1D -> Relu -> BatchNormalization -> dilated Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for _ in range(RESNET_2_BLOCKS):
      for dilation in DILATION:
        out_batchnorm = layers.BatchNormalization()(input_layer)
        out_conv1 = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding='same', activation='relu', dilation_rate=dilation)(out_batchnorm)
        out_batchnorm = layers.BatchNormalization()(out_conv1)
        out_conv1 = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding='same', activation='relu', dilation_rate=dilation)(out_batchnorm)
        input_layer = layers.Add()([input_layer, out_conv1])
    return input_layer

In [ ]:
def build_network():
    """
    builds the neural network architecture as shown in the exercise.
    :return: a Keras Model
    """
    ...
    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer)

    # # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding="same")(resnet_layer)

    # # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer)

    dropout_layer = layers.Dropout(DROPOUT)(resnet_layer)

    conv1d_layer = layers.Conv1D(np.ceil(RESNET_2_KERNEL_NUM/2), RESNET_2_KERNEL_SIZE, padding="same", activation='elu')(dropout_layer)

    output1 = layers.Dense(utils.OUTPUT_SIZE)(conv1d_layer)

    output2 = layers.Dense(1)(conv1d_layer)

    return tf.keras.Model(input_layer, [output1, output2], name='from_seq_to_struct')



In [ ]:
def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15,3))
    axes.plot(history.history['loss'][10:], label='Training loss')
    axes.plot(history.history['val_loss'][10:], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("/content/drive/MyDrive/Ex4Data/model_loss_history")  # TODO: you can change the path here


In [ ]:
def first_loss(y_true, y_pred):
    y_true = y_true[:,:,:-1]
    y_pred = y_pred[:,:,:-1]
    mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    return mse(y_true, y_pred)

def second_loss(y_true, y_pred):
    a = 0.01
    acc_pred = tf.squeeze(y_pred[:,:,-1]) # 32 X 140
    print(acc_pred.shape)
    y_true = y_true[:,:,:-1]
    y_pred = y_pred[:,:,:-1]
    mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    acc_val = mse(y_true, y_pred)
    return mse(y_true, y_pred) + mse(acc_val, acc_pred)

loss_mse_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)


def rmsd_loss(y_true, y_pred):
  mse_func = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
  mse = mse_func(y_true, y_pred)
  rmsd = tf.sqrt(((mse * 15) / 5))
  return rmsd 


def loss_acc_fn(y_true, y_pred, coef):
    return coef * rmsd_loss(y_true, y_pred)



In [ ]:
model = build_network()
# Instantiate an optimizer.
optimizer= tf.keras.optimizers.Adam(learning_rate=LR)
# Instantiate a loss function.

loss_fn = first_loss

# Prepare the training dataset.
# you can load here your input and output data
X = np.load('input_data.npy')
Y = np.load('label_data.npy')
# Y = np.concatenate([Y, np.zeros((Y.shape[0],Y.shape[1],1)).astype('float64') ], -1)

X_train, X_test, y_train, y_test  = train_test_split(X, Y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(BATCH)

coef = 0
epochs = 60
for epoch in range(epochs):
    
    if epoch == 10:
      coef = 0.03

    print("\nepoch %d" % (epoch,))
    start_time = time.time()
    loss_value = 0
    i = 1
    train_mse_losses = []
    train_acc_losses = []
    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        i = step
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            y_pred, acc_pred = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            rmsd_loss_value = rmsd_loss(y_batch_train, tf.cast(y_pred, tf.float64))
            acc_loss_value = loss_acc_fn(mse_loss_value, tf.cast(acc_pred, tf.float64), coef)
            
            train_mse_losses.append(rmsd_loss_value)
            train_acc_losses.append(acc_loss_value)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient([mse_loss_value,acc_loss_value], model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    ##### end of epoch #######
    total_mse_loss = tf.reduce_mean(tf.add_n(train_mse_losses[:-1])) /(i-1)
    total_acc_loss = tf.reduce_mean(tf.add_n(train_acc_losses[:-1])) /(i-1)


    # Run a validation loop at the end of each epoch.
    mse_val_losses = []
    acc_val_losses = []
    j = 1
    for k, (x_batch_val, y_batch_val) in enumerate(val_dataset):
        j = k
        val_preds, val_acc_pred = model(x_batch_val, training=False)
        # Update val metrics
        mse_val_loss_value = loss_mse_fn(y_batch_val, tf.cast(val_preds, tf.float64))
        acc_val_loss_value = loss_acc_fn(mse_val_loss_value, tf.cast(val_acc_pred, tf.float64), coef)
        mse_val_losses.append(mse_val_loss_value)
        acc_val_losses.append(acc_val_loss_value)

    total_val_mse_loss = tf.reduce_mean(tf.add_n(mse_val_losses[:-1])) /(j-1)
    total_val_acc_loss = tf.reduce_mean(tf.add_n(acc_val_losses[:-1])) /(j-1)

    print("MSE LOSS: train: %.4f, val: %.4f" % (float(total_mse_loss,), float(total_val_mse_loss,)))
    print("ACC LOSS: train: %.4f, val: %.4f" % (float(total_acc_loss,), float(total_val_acc_loss,)))

    if (total_val_mse_loss < 1) and (total_val_acc_loss<0.8) and epoch >11:
      break


epoch 0
MSE LOSS: train: 8.3061, val: 48.1791
ACC LOSS: train: 0.0000, val: 0.0000

epoch 1
MSE LOSS: train: 2.9342, val: 15.3395
ACC LOSS: train: 0.0000, val: 0.0000

epoch 2
MSE LOSS: train: 2.2905, val: 9.4681
ACC LOSS: train: 0.0000, val: 0.0000

epoch 3
MSE LOSS: train: 2.0860, val: 5.3418
ACC LOSS: train: 0.0000, val: 0.0000

epoch 4
MSE LOSS: train: 1.9360, val: 3.2517
ACC LOSS: train: 0.0000, val: 0.0000

epoch 5
MSE LOSS: train: 1.8128, val: 3.6766
ACC LOSS: train: 0.0000, val: 0.0000

epoch 6
MSE LOSS: train: 1.7760, val: 2.0615
ACC LOSS: train: 0.0000, val: 0.0000

epoch 7
MSE LOSS: train: 1.6728, val: 1.8735
ACC LOSS: train: 0.0000, val: 0.0000

epoch 8
MSE LOSS: train: 1.6221, val: 1.3680
ACC LOSS: train: 0.0000, val: 0.0000

epoch 9
MSE LOSS: train: 1.5813, val: 1.3543
ACC LOSS: train: 0.0000, val: 0.0000

epoch 10
MSE LOSS: train: 1.5776, val: 1.3981
ACC LOSS: train: 0.2687, val: 0.1849

epoch 11
MSE LOSS: train: 1.5330, val: 1.3442
ACC LOSS: train: 0.0514, val: 0.1677


In [ ]:
to_predict = utils.generate_input(os.path.join("/content/drive/MyDrive/Ex4Data", "6xw6.pdb"))
prediction = model.predict(to_predict[np.newaxis,:,:])
model.save("model_hackathon.tf")


INFO:tensorflow:Assets written to: model_hackathon.tf/assets


In [ ]:
prediction[1]

array([[[1.9448955 ],
        [1.7418382 ],
        [1.363431  ],
        [1.0181375 ],
        [0.9934501 ],
        [0.9922468 ],
        [1.2001047 ],
        [1.2456703 ],
        [1.6239715 ],
        [1.6880713 ],
        [1.8320215 ],
        [1.9995065 ],
        [1.5750282 ],
        [1.6226351 ],
        [1.4919109 ],
        [1.3211989 ],
        [1.3873692 ],
        [1.1898785 ],
        [1.0811436 ],
        [0.9769029 ],
        [0.81213206],
        [0.9034911 ],
        [1.0543442 ],
        [1.6187351 ],
        [1.8509829 ],
        [1.4220653 ],
        [1.3730133 ],
        [1.3425877 ],
        [1.6506405 ],
        [1.7240767 ],
        [1.3510964 ],
        [1.2944655 ],
        [0.89131576],
        [0.75202096],
        [0.5126294 ],
        [0.7997649 ],
        [0.8475759 ],
        [1.2593007 ],
        [1.1350553 ],
        [1.4036691 ],
        [1.7965639 ],
        [1.322655  ],
        [1.2428031 ],
        [1.1876031 ],
        [1.1327151 ],
        [1

0.05
MSE LOSS: train: 0.7906, val: 1.1091
ACC LOSS: train: 0.0585, val: 1.5253

In [ ]:
# if __name__ == '__main__':
#     model = build_network()
#     print(model.summary())

#     # you can load here your input and output data
#     X = np.load('input_data.npy')
#     Y = np.load('label_data.npy')

#     X_train, X_val, y_train, y_val  = train_test_split(X, Y, test_size=0.2, random_state=1)

#     my_optimizer= tf.keras.optimizers.Adam(learning_rate=LR)
#     model.compile(optimizer=my_optimizer, loss="mse")
#     history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH, validation_data=(X_val, y_val))

#     model.save("our_model.tf")
#     model = tf.keras.models.load_model("our_model.tf")

#     # part 3 predict
#     to_predict = utils.generate_input(os.path.join("/content/drive/MyDrive/Ex4Data", "6xw6.pdb"))
#     prediction = model.predict(to_predict[np.newaxis,:,:])
#     matrix = utils.matrix_to_pdb(utils.get_seq_aa(os.path.join("/content/drive/MyDrive/Ex4Data", "6xw6.pdb"), "H")[0], prediction[0, :, :],"our_6xw6")


In [4]:
model = tf.keras.models.load_model("model_hackathon.tf")
to_predict = utils.generate_input(os.path.join("/content/drive/MyDrive/Ex4Data/NbTestSet", "1fns.pdb"))
struct, acc = model.predict(to_predict[np.newaxis,:,:])
matrix = utils.matrix_to_pdb(utils.get_seq_aa(os.path.join("/content/drive/MyDrive/Ex4Data/NbTestSet", "1fns.pdb"), "H")[0], struct[0, :, :],"hackathon_result/test_1fns")


In [34]:
import glob
model = tf.keras.models.load_model("our_model.tf")
test_files_path = '/content/drive/MyDrive/Ex4Data/NbTestSet'
result_path = '/content/drive/MyDrive/Ex4Data/hackathon_result_ex4_model/'
for file in glob.glob(test_files_path + "/*.pdb"):
    protein_name = os.path.basename(os.path.splitext(file)[0])
    to_predict = utils.generate_input(file)
    struct = model.predict(to_predict[np.newaxis,:,:])
    utils.matrix_to_pdb(utils.get_seq_aa(file, "H")[0], struct[0, :, :], result_path + protein_name)
    # np.savetxt(result_path + protein_name + '.txt', np.squeeze(acc), fmt='%5f')



In [28]:
result_path = '/content/drive/MyDrive/Ex4Data/hackathon_result/'

np.savetxt(result_path+'tewst_lacc.txt', np.squeeze(acc), fmt='%5f')

In [22]:
import sys

def main(args):
  pdb_path = args[1]
  method = args[2]
  run_prediction(pdb_path, method)

def run_prediction(pdb_path, method):
  if method == "baseline":
    pass
    
  if method == "advanced":
    path_to_advanced_model = "model_hackathon.tf"
    model = tf.keras.models.load_model(path_to_advanced_model)
    protein_name = os.path.basename(os.path.splitext(pdb_path)[0])
    to_predict = utils.generate_input(pdb_path)
    struct, acc = model.predict(to_predict[np.newaxis,:,:])
    utils.matrix_to_pdb(utils.get_seq_aa(pdb_path, "H")[0], struct[0, :, :], protein_name)
    np.savetxt(protein_name + '_acc.txt', np.squeeze(acc), fmt='%5f')
  

if __name__ == '__main__':
    main(sys.argv)



array([2.4384, 2.025 , 1.683 , 1.366 , 1.0191, 0.9663, 1.0071, 1.2155,
       1.3361, 1.668 , 1.6741, 1.8112, 1.9966, 1.523 , 1.6029, 1.6045,
       1.3444, 1.3998, 1.2183, 1.067 , 0.9854, 0.8155, 0.932 , 1.0925,
       1.6575, 1.918 , 1.364 , 1.4077, 1.2813, 1.7023, 1.7984, 1.3554,
       1.293 , 0.8457, 0.6878, 0.4848, 0.7971, 0.8097, 1.1728, 1.1113,
       1.338 , 1.8374, 1.3732, 1.3394, 1.1407, 1.1354, 1.2245, 1.1508,
       1.2142, 1.2582, 1.3359, 1.7216, 1.8776, 2.1951, 2.4192, 2.303 ,
       2.0503, 1.6735, 1.7282, 1.5102, 1.4842, 1.4883, 1.4509, 1.6203,
       1.4305, 1.452 , 1.4408, 1.6735, 1.1348, 1.0274, 0.8618, 0.9096,
       0.9396, 1.08  , 1.1387, 1.5991, 1.6655, 1.893 , 1.7262, 1.246 ,
       1.2356, 1.1711, 0.7154, 1.2049, 0.6927, 1.0941, 0.9539, 1.1305,
       1.2766, 1.6147, 1.5652, 1.345 , 1.5202, 1.109 , 1.2723, 0.9425,
       1.0398, 1.1706, 1.2455, 1.5054, 1.7218, 2.1962, 2.4921, 2.5143,
       2.3279, 1.9989, 2.1087, 1.7841, 1.8378, 1.635 , 1.725 , 1.2873,
      